In [1]:
from dariah_topics import preprocessing as pre
from dariah_topics import visualization as visual
from dariah_topics import mallet as mal
# Warning is Gensim related

c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  spec = inspect.getargspec(func)
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\funcy\decorators.py:56: DeprecationW

## Preprocessing

#### Liste mit Dateinamen erzeugen

In [2]:
path_txt = "corpus_txt"
#path_txt = "grenzbote_plain/*/"

doclist_txt = pre.create_document_list(path_txt)
doclist_txt[:5]

c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\formatters.py:92: DeprecationWarning: DisplayFormatter._ipython_display_formatter_default is deprecated: use @default decorator instead.
  def _ipython_display_formatter_default(self):
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\formatters.py:98: DeprecationWarning: DisplayFormatter._formatters_default is deprecated: use @default decorator instead.
  def _formatters_default(self):
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\formatters.py:677: DeprecationWarning: PlainTextFormatter._deferred_printers_default is deprecated: use @default decorator instead.
  def _deferred_printers_default(self):
c:\users\philip\appdata\local\programs\python\python35-32\lib\site-packages\IPython\core\formatters.py:669: DeprecationWarning: PlainTextFormatter._singleton_printers_default is deprecated: use @default decorator instead.

['corpus_txt\\Doyle_AScandalinBohemia.txt',
 'corpus_txt\\Doyle_AStudyinScarlet.txt',
 'corpus_txt\\Doyle_TheHoundoftheBaskervilles.txt',
 'corpus_txt\\Doyle_TheSignoftheFour.txt',
 'corpus_txt\\Howard_GodsoftheNorth.txt']

In [3]:
path_csv = "corpus_csv"

doclist_csv = pre.create_document_list(path_csv, 'csv')
doclist_csv[:5]

['corpus_csv\\Doyle_AStudyinScarlet.txt.csv',
 'corpus_csv\\Doyle_TheHoundoftheBaskervilles.txt.csv',
 'corpus_csv\\Doyle_TheSignoftheFour.txt.csv',
 'corpus_csv\\Howard_GodsoftheNorth.txt.csv',
 'corpus_csv\\Howard_SchadowsinZamboula.txt.csv']

#####  Liste mit Dokumentenlabels erzeugen - (Funktion wird durch Thorsten's generischere Funktion ersetzt)

In [4]:
doc_labels = list(pre.get_labels(doclist_txt))
doc_labels[:5]

['corpus_txt\\Doyle_AScandalinBohemia.txt',
 'corpus_txt\\Doyle_AStudyinScarlet.txt',
 'corpus_txt\\Doyle_TheHoundoftheBaskervilles.txt',
 'corpus_txt\\Doyle_TheSignoftheFour.txt',
 'corpus_txt\\Howard_GodsoftheNorth.txt']

#### Corpus laden

In [5]:
corpus_txt = pre.read_from_txt(doclist_txt)

In [6]:
corpus_csv = pre.read_from_csv(doclist_csv)

#### Tokenisieren

In [7]:
doc_tokens = [list(pre.tokenize(txt)) for txt in list(corpus_txt)]
doc_tokens[:1]

[['a',
  'scandal',
  'in',
  'bohemia',
  'a',
  'conan',
  'doyle',
  'i',
  'to',
  'sherlock',
  'holmes',
  'she',
  'is',
  'always',
  'the',
  'woman',
  'i',
  'have',
  'seldom',
  'heard',
  'him',
  'mention',
  'her',
  'under',
  'any',
  'other',
  'name',
  'in',
  'his',
  'eyes',
  'she',
  'eclipses',
  'and',
  'predominates',
  'the',
  'whole',
  'of',
  'her',
  'sex',
  'it',
  'was',
  'not',
  'that',
  'he',
  'felt',
  'any',
  'emotion',
  'akin',
  'to',
  'love',
  'for',
  'irene',
  'adler',
  'all',
  'emotions',
  'and',
  'that',
  'one',
  'particularly',
  'were',
  'abhorrent',
  'to',
  'his',
  'cold',
  'precise',
  'but',
  'admirably',
  'balanced',
  'mind',
  'he',
  'was',
  'i',
  'take',
  'it',
  'the',
  'most',
  'perfect',
  'reasoning',
  'and',
  'observing',
  'machine',
  'that',
  'the',
  'world',
  'has',
  'seen',
  'but',
  'as',
  'a',
  'lover',
  'he',
  'would',
  'have',
  'placed',
  'himself',
  'in',
  'a',
  'false'

#### Create Dictionaries

In [8]:
id_types, doc_ids = pre.create_dictionaries(doc_labels, doc_tokens)

#### Sparse BOW Model

In [9]:
sparse_bow = pre.create_mm(doc_labels, doc_tokens, id_types, doc_ids)

In [10]:
sparse_bow

0
doc_id token_id    
0      16387      1
       6          1
       20488      3
       8201       1
       16398     25
       8207       1
       8208       1
       8215       3
       1894       1
       20506      2
       20509      3
       12318      1
       31         1
       8225       2
       8226       1
       16424      1
       20523      1
       12978      1
       20526      2
       12337      1
       4147       1
       20533      1
       20534      1
       55         1
       8249       2
       11948      1
       16446      7
       12353      4
       66         3
       8262       2
...              ..
16     24507      2
       23882      1
       8126       1
       16319      1
       23457      2
       24516      1
       20421      1
       16327      1
       8136       1
       5452       1
       20427      1
       4052       1
       23886      1
       16343      1
       12248      1
       4060       1
       16378      3
       16352      1
       24548      1
       2726       1
       20455      1
       2727       6
       7583       5
       12270      3
       16370      1
       17747      1
       20469      1
       20471      1
       3273       1
       8190      17

[58745 rows x 1 columns]

##### Speichern der Market Matrix

In [11]:
'''num_docs = max(sparse_bow.index.get_level_values("doc_id"))
num_types = max(sparse_bow.index.get_level_values("token_id"))
sum_counts = sum(sparse_bow[0])

header_string = str(num_docs) + " " + str(num_types) + " " + str(sum_counts) + "\n"

with open("gb_plain.mm", 'a', encoding = "utf-8") as f:
    f.write("%%MatrixMarket matrix coordinate real general\n")
    #f.write(header_string)
    sparse_bow.to_csv( f, sep = ' ', header = None)'''

NameError: name 'sparse_df' is not defined

#### Remove Features

In [ ]:
import os.path
basepath =  os.path.abspath(os.path.join(".", os.pardir))

with open(os.path.join(basepath, "Topics/tutorial_supplementals/stopwords/en"), 'r', encoding = 'utf-8') as f: 
    stopword_list = f.read().split('\n')
    
stopword_list = set(stopword_list)

In [ ]:
sparse_df_stopwords_removed = pre.remove_features(sparse_bow, id_types, stopword_list)

In [ ]:
print(len(sparse_bow))
print(len(sparse_df_stopwords_removed))

##### Create Feature Remove Lists

In [ ]:
stopword_list = pre.find_stopwords(docterm_matrix, 100)
stopword_list

In [ ]:
hapax = set(docterm_matrix[docterm_matrix == 1].index)
#hapax = set(docterm_matrix[(docterm_matrix < 2).all(axis=1)].index)
len(hapax)

In [ ]:
hapax_list = collection.find_hapax(docterm_matrix)
len(hapax_list)

In [ ]:
feature_list = stopword_list.union(hapax_list)
print(feature_list)
clean_term_frequency = docterm_matrix.drop(feature_list)

##### Remove Features

In [ ]:
feature_list = stopword_list.union(hapax_list)
clean_term_frequency = collection.remove_features(docterm_matrix, feature_list)

##### Lemmata anhand POS-Tags auswählen

In [ ]:
lemmas = collection.filter_POS_tags(corpus_csv)
next(lemmas)

## Topic Modeling with Gensim

In [ ]:
corpusPath = "gb_plain.mm"

mm = MmCorpus(corpusPath) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen


In [ ]:
#in case you're only loading the corpus - build dict first

import pickle

pickle.dump( id_types, open( "gb_plain.dictionary", "wb" ) )

In [ ]:
id_types = pickle.load(open("gb_plain.dictionary", 'rb'))

In [ ]:
#mm = gensim.corpora.MmCorpus("gb_all.mm")

In [ ]:
#id_types = pickle.load(open("gb_all.dictionary", 'rb'))

In [ ]:
model = LdaModel(corpus=mm, id2word=id_types, num_topics=60, alpha = "symmetric", passes = 10) #import momentan in visual 
# -> da ich mir noch nicht sicher bin, welche Funktionen in das tm_gensim.py sollen

In [ ]:
topic_nr_x = model.get_topic_terms(10)

[id_types[i[0]] for i in topic_nr_x]

In [ ]:
topics = model.show_topics(num_topics = 60)

In [ ]:
topics

## Topic Modeling mit Mallet

##### Mallet Binary erzeugen

In [ ]:
import os.path
basepath =  os.path.abspath(os.path.join(".", os.pardir))
basepath

In [ ]:
#path_to_mallet = "/home/sina/Uni/Dariah/mallet/bin/mallet"
path_to_mallet = "mallet"
path_to_corpus = os.path.join(basepath, "Topics\\corpus_txt")

malletBinary, outfolder = mal.create_mallet_binary(path_to_corpus, path_to_mallet)

#toDo absolute Pfade

##### Mallet output erzeugen

In [ ]:
path_to_mallet = "/home/sina/Uni/Dariah/mallet/bin/mallet"
malletBinary = "/home/sina/Uni/Dariah/DARIAH-Topics/Topics/mallet_output/malletBinary.mallet"
mal.create_mallet_model(malletBinary, outfolder, path_to_mallet)

## Visualisierung 

In [ ]:
lda_model = 'out_easy/corpus.lda'
corpus = 'out_easy/corpus.mm'
dictionary = 'out_easy/corpus.dict'
doc_labels = 'out_easy/corpus_doclabels.txt'
interactive  = False

vis = visual.Visualization(lda_model, corpus, dictionary, doc_labels, interactive)

#### Heatmap

In [ ]:
heatmap = visual.make_heatmap()

In [ ]:
visual.save_heatmap("./visualizations/heatmap")

#### Interactive

In [ ]:
vis = collection.Visualization(lda_model, corpus, dictionary, doc_labels, interactive=True)

In [ ]:
vis.make_interactive()

In [ ]:
vis.save_interactive("./visualizations/interactive")

#### Ob ihr wirklich richtig steht, seht ihr, wenn ...

In [ ]:
![success](http://cdn2.hubspot.net/hub/128506/file-446943132-jpg/images/computer_woman_success.jpg)